# Рекомендательные системы. Матричные разложения

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split


## Load dataset

Мы будем использовать тот же датасет из прошлого урока - MovieLens.

In [ ]:
df_ratings = pd.read_csv("ratings.csv")
df_movies = pd.read_csv("movies.csv")

In [ ]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### SVD

In [ ]:
from scipy.linalg import svd

In [ ]:
# define a matrix
A = np.array([[1, 2], [3, 4], [5, 6]]) #np.nan
print(A)

[[1 2]
 [3 4]
 [5 6]]


In [ ]:
# SVD
U, s, VT = svd(A) # Factorizes the matrix a into two unitary matrices U and VT, and a 1-D array s of singular values (real, non-negative) such that a == U @ S @ Vh, where S is a suitably shaped matrix of zeros with main diagonal s
print("Unitary matrix having left singular vectors as columns: \n", U) # U*s is the length of projections on the axes (by columns)
print("Singular values: \n", s) # this is diagonal matrix with sigmas on the diagonal (closeness to tthe axes)
print("Unitary matrix having right singular vectors as rows: \n", VT) # ortogonal basis of the projection

Unitary matrix having left singular vectors as columns: 
 [[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
Singular values: 
 [9.52551809 0.51430058]
Unitary matrix having right singular vectors as rows: 
 [[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


In [ ]:
m, n = A.shape[0], A.shape[1]

sigma = np.zeros((m, n))
for i in range(min(m, n)):
    sigma[i, i] = s[i]
A_rec = np.dot(np.dot(U,sigma), VT)
print("Reconstructed matrix: \n", A_rec)

Reconstructed matrix: 
 [[1. 2.]
 [3. 4.]
 [5. 6.]]


## Train/Test split

In [ ]:
import scipy.sparse as sp
from scipy import sparse
from scipy.sparse.linalg import spsolve

In [ ]:
user_item_matrix = df_ratings.pivot_table(index=['userId'], columns=['movieId'], values='rating')
# Contruct a sparse matrix for our users and items containing number of plays
sparse_ui= sp.csr_matrix(user_item_matrix)

In [ ]:
sparse_ui.todense()

matrix([[4. , nan, 4. , ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [2.5, 2. , 2. , ..., nan, nan, nan],
        [3. , nan, nan, ..., nan, nan, nan],
        [5. , nan, nan, ..., nan, nan, nan]])

In [ ]:
X_train, X_test = train_test_split(sparse_ui, test_size = 0.25, random_state=57)
ind_train, ind_test = train_test_split(user_item_matrix, test_size = 0.25, random_state=57)

In [ ]:
X_res = pd.DataFrame(index=ind_train.index, data=[], columns=['actual'])
for i in X_res.index:
    X_res.loc[i]['actual'] = list(set(ind_train.loc[i][ind_train.loc[i].notnull()].index))

<ipython-input-21-45306e47ae90>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  X_res.loc[i]['actual'] = list(set(ind_train.loc[i][ind_train.loc[i].notnull()].index))


In [ ]:
X_res.head()

,actual
userId,
575,"[2560, 2436, 2566, 2567, 2568, 2571, 2572, 257..."
323,"[1, 2, 2571, 1037, 527, 17, 19, 22, 110102, 29..."
14,"[4, 7, 266, 524, 527, 784, 19, 150, 153, 25, 2..."
496,"[4993, 111362, 2950, 904, 104841, 912, 84374, ..."
531,"[4993, 260, 10, 1291, 919, 1961, 2473, 1198, 1..."


In [ ]:
ind_train_u = pd.Series(ind_train.index.tolist())
ind_train_i = pd.Series(ind_train.columns.values.tolist())

In [ ]:
print(X_train.shape, X_test.shape)

(457, 9724) (153, 9724)


### Latent Factor Model ()

Будем искать значение целовой переменной $r_{ui}$ пользователя $u$, поставленная фильму $i$, как скалярное произведение векторов $p_u$ и $q_i$ в некотором пространстве $T$ латентных признаков:
$$
    r_{ui}
    \approx
    \langle p_u, q_i \rangle.
$$


Как мы обсуждали на уроке нам необходимо минимизировать следующий функционал (LFM):

\begin{equation}
\label{eq:lfmReg}
    \sum_{(u, i) \in R}
        \left(
            r_{ui}
            - \langle p_u, q_i \rangle
        \right)^2
    +
    \lambda
    \sum_{u \in U}
        \|p_u\|^2
    +
    \mu
    \sum_{i \in I}
        \|q_i\|^2
    \to
    \min_{P, Q}
\end{equation}

Мы проведем $N$ итераций, в рамках каждой итерации сначала оптимизируем $p$ при фиксированном
$q$, затем $q$ при фиксированном $p$.

Составим матрицу $P$ из векторов $p_u$ и матрицу $Q$ из векторов $q_i$. Матрицей $Q[u] \in R^{n_u×K}$ будем обозначать подматрицу матрицы $Q$ только для товаров, оцененных пользователем $u$, где $n_u$ – количество оценок пользователя $u$.
Шаг перенастройки $p_u$ при фиксированной матрице $Q$ сводится к настройке Ridge-регрессии и выглядит так:
$$A_u = Q[u]^T Q[u] $$
$$d_u = Q[u]^Tr_u $$
$$p_u = (\lambda n_uI + A_u)^{−1}d_u
$$

Есть __явные (explicit)__ предпочтения, которые более надежно дают характеристику подходящего объекта для пользователей:
- Покупки, добавления в корзину
- Рейтинги

И __неявные (implicit)__, их как правило больше:
- Посещение страницы товара
- Просмотр части фильма

__Идея iALS__: предсказываем $s_{ui}$ c весом $c_{ui} = 1 + \alpha*r_{ui}$. Коэффициент $\alpha$ позволяет регулировать влияние явного рейтинга на уверенность в интересе.

\begin{equation}
\label{eq:lfmReg}
    \sum_{(u, i) \in R}
        \left(c_{ui}*(
            s_{ui}
            - \langle p_u, q_i \rangle
        \right)^2)
    +
    \lambda
    \sum_{u \in U}
        \|p_u\|^2
    +
    \mu
    \sum_{i \in I}
        \|q_i\|^2
    \to
    \min_{P, Q}
\end{equation}

In [ ]:
from scipy import sparse
from scipy.sparse.linalg import spsolve
# from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [ ]:
seed=57
rstate = np.random.RandomState(seed)
rank_size=10
lambda_val = 0.1
num_user = X_train.shape[0]
num_item = X_train.shape[1]
P = sparse.csr_matrix((rstate.normal(size = (num_user, rank_size)))) # Random numbers in a m x rank shape
Q = sparse.csr_matrix((rstate.normal(size = (num_item, rank_size)))) # Normally this would be rank x n but we can transpose at the end. Makes calculation more simple.
QTQ = Q.T.dot(Q) # QTQ
PTP = P.T.dot(P)
P_eye = sparse.eye(num_user)
Q_eye = sparse.eye(num_item)
lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I.

In [ ]:
u=5
pref = X_train[u, :].toarray() # Grab user row from confidence matrix and convert to dense
pref_u = pref[~np.isnan(pref)] # We take only the movies which the user has rated
u_rated_movies_ind = np.argwhere(~np.isnan(pref))[:,1] # Index of the rated movies
Qu = Q[u_rated_movies_ind, :] # We construct the Qu matrix of only rated existant pairs (u, i)
QuTru = Qu.T.dot(pref_u.T) # This is the QuTPu term
Q[u] = spsolve(QTQ +lambda_eye, QuTru)
print(Q[u].toarray())

[[ 0.0044179  -0.01638758 -0.00766912 -0.00914071  0.02136258 -0.00095192
  -0.00905147  0.01610489  0.00381576 -0.00747813]]


In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse import eye

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, eye
from scipy.sparse.linalg import spsolve

def lfm_als(training_set, lambda_val, iterations=10, rank_size=20, seed=57):
    """
    Alternating Least Squares (ALS) implementation for Latent Factor Model.

    Parameters:
    - training_set: Sparse CSR matrix of shape (num_users x num_items) containing user-item ratings.
    - lambda_val: Regularization parameter to control overfitting.
    - iterations: Number of ALS iterations to run (default is 10).
    - rank_size: Number of latent features (default is 20).
    - seed: Random seed for reproducibility (default is 57).

    Returns:
    - P: User feature matrix.
    - Q.T: Transposed item feature matrix.
    """

    # Get the size of the ratings matrix
    num_user = training_set.shape[0]
    num_item = training_set.shape[1]

    # Initialize random state and feature matrices P and Q
    rstate = np.random.RandomState(seed)
    P = rstate.normal(size=(num_user, rank_size))  # User feature matrix
    Q = rstate.normal(size=(num_item, rank_size))  # Item feature matrix

    lambda_eye = lambda_val * eye(rank_size)  # Regularization term

    # Begin ALS iterations
    for iter_step in range(iterations):
        # Precompute Q^T * Q and P^T * P for efficiency
        QTQ = Q.T.dot(Q)
        PTP = P.T.dot(P)

        # Update user feature matrix P
        for u in range(num_user):
            pref_u = training_set[u, :].toarray().flatten()  # User's preferences as a dense array
            rated_items_indices = np.where(pref_u > 0)[0]   # Indices of rated items

            if len(rated_items_indices) > 0:
                Qu = Q[rated_items_indices, :]             # Subset of Q corresponding to rated items
                QuTru = Qu.T.dot(pref_u[rated_items_indices])  # Compute Qu^T * r_u

                P[u] = spsolve(QTQ + lambda_eye, QuTru)     # Solve for user features

        # Update item feature matrix Q
        for i in range(num_item):
            pref_i = training_set[:, i].toarray().flatten()  # Item's preferences as a dense array
            rated_users_indices = np.where(pref_i > 0)[0]   # Indices of users who rated this item

            if len(rated_users_indices) > 0:
                Pu = P[rated_users_indices, :]             # Subset of P corresponding to users who rated this item
                PuTri = Pu.T.dot(pref_i[rated_users_indices])  # Compute Pu^T * r_i

                Q[i] = spsolve(PTP + lambda_eye, PuTri)     # Solve for item features

    return csr_matrix(P), csr_matrix(Q.T)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Выполним факторизацию матрицы рейтингов training_set с помощью lfm_als
P, Q_T = lfm_als(X_train, lambda_val=0.1)

# Транспонируем Q_T обратно в исходную матрицу признаков фильмов (размер n x rank_size)
Q = Q_T.T.toarray()

# Рассчитаем косинусное сходство между всеми фильмами
similarity_matrix = cosine_similarity(Q)

# Найдем индексы фильмов, похожих на фильм с movieId=5615
movie_index = 5615  # Индекс фильма в матрице (зависит от структуры данных)
similar_movies_indices = similarity_matrix[movie_index].argsort()[::-1]  # Сортировка по убыванию сходства

# Второй фильм в списке похожих (первый — это сам фильм)
second_similar_movie_index = similar_movies_indices[1]

print("ID второго похожего фильма:", second_similar_movie_index)


<ipython-input-60-4bda2c4519de>:47: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  P[u] = spsolve(QTQ + lambda_eye, QuTru)     # Solve for user features
<ipython-input-60-4bda2c4519de>:58: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  Q[i] = spsolve(PTP + lambda_eye, PuTri)     # Solve for item features


ID второго похожего фильма: 6245


In [ ]:
%%time
P, Q = lfm_als(X_train, lambda_val = 0.1, iterations = 10, rank_size = 20)

CPU times: user 12min 42s, sys: 1 s, total: 12min 43s
Wall time: 12min 47s


In [ ]:
print("P shape: {}".format(P.shape))
print("Q shape: {}".format(Q.shape))

P shape: (457, 20)
Q shape: (20, 9724)


### Find Similar movies

Now that we have the embeddings for all the movies and all the users, we can find similar movies

In [ ]:
df_movies[df_movies['title'].str.contains('Shrek')]

,movieId,title,genres
3194,4306,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...
5160,8360,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...
6486,53121,Shrek the Third (2007),Adventure|Animation|Children|Comedy|Fantasy
6915,64249,Shrek the Halls (2007),Adventure|Animation|Comedy|Fantasy
7360,78637,Shrek Forever After (a.k.a. Shrek: The Final C...,Adventure|Animation|Children|Comedy|Fantasy|IMAX


In [ ]:
# Let's find similar movies to Shrek (2001).
movieId = 5615
ind_i = ind_train_i[ind_train_i == movieId].index[0]

# Get the item row
qi = Q[:, ind_i].toarray()[:, 0]

In [ ]:
# Calculate the similarity score between choseen movie and other movies
# and select the top 10 most similar.
scores = Q.T.dot(qi)
top_10 = np.argsort(scores)[::-1][:10]

In [ ]:
top_10

array([  20,  509, 1796,  398,  505,  508,  514,  315, 1001,  383])

In [ ]:
# since we have indexes are not equal to movieId, we take the movieId from the
df_movies[df_movies['movieId']==ind_train_i[3828]]['title'].iloc[0]

'Believer, The (2001)'

In [ ]:
movies = []
movies_genres = []
movies_scores = []
movies_ids = []

# Get and print the actual artists names and scores
for idx in top_10:
    movies_ids.append(ind_train_i[idx])
    movies.append(df_movies[df_movies['movieId']==ind_train_i[idx]]['title'].iloc[0])
    movies_genres.append(df_movies[df_movies['movieId']==ind_train_i[idx]]['genres'].iloc[0])
    movies_scores.append(scores[idx])

similar = pd.DataFrame({'movieId': movies_ids, 'movies': movies, 'score': movies_scores, 'genres': movies_genres})
similar

,movieId,movies,score,genres
0,21,Get Shorty (1995),9929.793257,Comedy|Crime|Thriller
1,592,Batman (1989),9250.268785,Action|Crime|Thriller
2,2396,Shakespeare in Love (1998),8946.338297,Comedy|Drama|Romance
3,457,"Fugitive, The (1993)",8636.084672,Thriller
4,587,Ghost (1990),8452.765989,Comedy|Drama|Fantasy|Romance|Thriller
5,590,Dances with Wolves (1990),8436.466614,Adventure|Drama|Western
6,597,Pretty Woman (1990),8197.353452,Comedy|Romance
7,357,Four Weddings and a Funeral (1994),8029.325146,Comedy|Romance
8,1304,Butch Cassidy and the Sundance Kid (1969),7775.032965,Action|Western
9,440,Dave (1993),7771.717664,Comedy|Romance


### Recommend movies to the user

Let's write a function to predict for every user a movie not watched.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def predict_top_k(user_id, training_set, P, Q, df_movies, ind_train_i, k=10):
    """
    Recommend items for a given user given a trained model.

    Args:
        user_id (int): The id of the user we want to create recommendations for.
        training_set (csr_matrix): Our original training data.
        P (numpy.ndarray): User embedding matrix.
        Q (numpy.ndarray): Item embedding matrix.
        df_movies (pd.DataFrame): DataFrame containing movie information (e.g., movieId and title).
        ind_train_i (list): List mapping training matrix indices to movieIds.
        k (int): Number of recommendations to return.

    Returns:
        recommendations (pd.DataFrame): DataFrame with k movies and scores.
    """
    # Проверяем индекс пользователя для соответствия матрице
    matrix_user_id = user_id - 1  # Если ID пользователей начинаются с 1

    # Проверка размеров P и Q
    print("Размер P:", P.shape)
    print("Размер Q:", Q.shape)

    # Транспонирование Q для корректного скалярного произведения
    if Q.shape[0] != P.shape[1]:
        print("Транспонируем Q...")
        Q = Q.T

    # Вычисление рекомендаций через скалярное произведение
    rec_vector = P[matrix_user_id, :].dot(Q)

    # Получаем взаимодействия пользователя
    user_interactions = training_set[user_id, :].toarray()

    # Бинарная маска: установим просмотренные фильмы в 0, а непросмотренные в 1
    user_interactions = np.where(~np.isnan(user_interactions), 0, user_interactions)
    user_interactions = np.nan_to_num(user_interactions[0], nan=1)

    # Вычисление рекомендаций через скалярное произведение
    rec_vector = P[matrix_user_id, :].dot(Q)  # Убираем .toarray()

    # Масштабируем оценки между 0 и 1 для удобства интерпретации
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1, 1))[:, 0]

    # Исключаем уже просмотренные фильмы
    recommend_vector = user_interactions * rec_vector_scaled

    # Получаем индексы топ-K фильмов в порядке убывания оценок
    item_idx = np.argsort(recommend_vector)[::-1][:k]

    # Подготовка DataFrame с рекомендациями
    movies = []
    movies_scores = []
    movies_ids = []

    for idx in item_idx:
        movies_ids.append(ind_train_i[idx])
        movie_title = df_movies[df_movies['movieId'] == ind_train_i[idx]]['title'].iloc[0]
        movies.append(movie_title)
        movies_scores.append(recommend_vector[idx])

    similar = pd.DataFrame({'movieId': movies_ids, 'title': movies, 'score': movies_scores})

    return similar


In [ ]:
# Let's generate and print our recommendations
user_id = 103
recommendations = predict_top_k(user_id, X_train, P, Q, df_movies, ind_train_i, k=10)
print(recommendations)

NameError: name 'X_train' is not defined

In [ ]:
# Let's put it in a list of recommendation for every user
pred = []
for i in X_res.index:
    ind = ind_train_u[ind_train_u == i].index[0] # get index in the sparse matrix
    recom = predict_top_k(ind, X_train, P, Q, df_movies, ind_train_i, k=10)
    pred.append(recom['movieId'].to_list())

In [ ]:
X_res['predicted'] = pred

In [ ]:
X_res.head()

,actual,predicted
userId,,
575,"[2560, 2436, 2566, 2567, 2568, 2571, 2572, 257...","[85025, 168250, 76060, 3758, 5991, 3790, 91542..."
323,"[1, 2, 2571, 1037, 527, 17, 19, 22, 110102, 29...","[103483, 32314, 6660, 8383, 81535, 37739, 2278..."
14,"[4, 7, 266, 524, 527, 784, 19, 150, 153, 25, 2...","[4403, 56941, 4022, 157172, 104074, 2028, 1350..."
496,"[4993, 111362, 2950, 904, 104841, 912, 84374, ...","[47538, 78160, 994, 72171, 64695, 4973, 3317, ..."
531,"[4993, 260, 10, 1291, 919, 1961, 2473, 1198, 1...","[6516, 4397, 567, 68952, 4293, 3719, 5247, 418..."
